# Double Pendulum

In [7]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['science', 'notebook', 'grid'])

In [8]:
class FCN(nn.Module):
    "Defines a connected network"

    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.ELU
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS - 1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)

    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

In [9]:
torch.manual_seed(15)
model = FCN(1, 2, 32, 3)

In [10]:
t_boundary = torch.tensor(0.).view(-1, 1).requires_grad_(True)
t_physics = torch.linspace(0, 1, 30).view(-1, 1).requires_grad_(True)
t_test = torch.linspace(0, 1, 300).view(-1, 1)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
num_iter = 15001

In [28]:
theta = model(t_boundary)
theta_p = torch.autograd.grad(theta, t_boundary, torch.ones_like(theta), create_graph=True)[0]
theta_p2 = torch.autograd.grad(theta_p, t_boundary, torch.ones_like(theta_p), create_graph=True)[0]
print(theta, '\n', theta_p, '\n', theta_p2)

tensor([[ 0.0665, -0.1339]], grad_fn=<AddmmBackward0>) 
 tensor([[-0.0612]], grad_fn=<TBackward0>) 
 tensor([[-0.0250]], grad_fn=<TBackward0>)


In [19]:
for i in range(num_iter):
    optimizer.zero_grad()

    theta = model(t_boundary)
    loss1 = (torch.squeeze(theta) - 60) ** 2
    lambda1 = 1

    theta_p = torch.autograd.grad(theta, t_boundary, torch.ones_like(theta), create_graph=True)[0]
    loss2 = (torch.squeeze(theta_p) - 0) ** 2
    lamda2 = 1e-4
    
    theta = model(t_physics)
    theta_p = torch.autograd.grad(theta, t_physics, torch.ones_like(theta), create_graph=True)[0]
    theta_pp = torch.autograd.grad(theta_p, t_physics, torch.ones_like(theta_p), create_graph=True)[0]
    loss3 = 
    lamda3 = 1e-4
    
    loss = loss1 + 1e-4 * loss2
    loss.backward()
    optimizer.step()

    if i % 2500 == 0:
        g = model(t_test).detach()
        diff = str(np.round(np.max(np.abs(g - g_exact).numpy()), 5))
        plt.figure(figsize=(8, 4))
        plt.scatter(t_physics.detach()[:, 0],
                    torch.zeros_like(t_physics)[:, 0], s=20, lw=0, color="tab:green", alpha=0.6)
        plt.scatter(t_boundary.detach()[:, 0],
                    torch.zeros_like(t_boundary)[:, 0], s=20, lw=0, color="tab:red", alpha=0.6)
        plt.plot(t_test[:, 0], g_exact[:, 0], label="Exact", color="tab:red", alpha=0.6)
        plt.plot(t_test[:, 0], g[:, 0], label="PINN", color="tab:green")
        plt.title(f"Training step {i}")
        plt.text(0.25, 9, f'max absolute difference: {diff}', size=10, bbox=dict(facecolor='white', edgecolor='black'))
        plt.legend()
        plt.show()

SyntaxError: invalid syntax (825590133.py, line 15)